# Sorting
By the end of this section you will be able to:
- sort a `DataFrame`
- sort a column with an expression 
- take advantage of fast-track algorithms with `set_sorted`

In this lecture we learn how to sort both on a `DataFrame` and within an expression. We also introduce the fast-track algorithms on sorted data. The fast-track algorithims are optimisations that can not be included as part of the built-in query optimiser so we show how to trigger them on simple problems here.

In [ ]:
import polars as pl

In [ ]:
csvFile = "../data/titanic.csv"

In [ ]:
df = pl.read_csv(csvFile)
df.head(3)

We use `pl.Config` to adjust the default so that only 4 rows of a `DataFrame` are printed

In [ ]:
pl.Config.set_tbl_rows(4)

## Sorting a `DataFrame`

### Using the `sort` method on `DataFrame`

We can sort a `DataFrame` on a column with the `sort` method

In [ ]:
df.sort("Age")

By default `null` values are at the start of the sort. 

We can move `null` values to the end with the `nulls_last` argument to `sort`.

We can sort based on multiple columns with a list

In [ ]:
df.sort(["Pclass","Age"])

## Sorting a column with an expression

We can transform a column into sorted order with an expression.

In this example we sort the values in every column independent of other columns

In [ ]:
(
    df
    .select(
        pl.all().sort()
    )
)

With the `sort` expression each column is sorted by itself and not with respect to another column.

Within an expression we can also sort all columns with respect to another column using `sort_by`

In [ ]:
(
    df
    .select(
        pl.all().sort_by("Age")
    )
)

It seems like `sort_by` in this case has just replicated the functionality of 
```python
df.sort("Age")
```
However, as we can use `sort_by` in an expression it can be used in other contexts such as in a `groupby` aggregation. 

## Taking advantage of sorted data

For some operations Polars can use a fast track algorithm if it knows the data in a column is sorted.

For example, if we want the `max` value on a sorted column we just take the last non-`null` value.

### Checking the sorted status
You can check if Polars thinks a column is sorted with the `flags` attribute

In [ ]:
df["PassengerId"].flags

In this case Polars doesn't think the `PassengerID` column is sorted (although it is sorted).

### Setting the sorted status
If we know that a column is sorted then we can let Polars know using `set_sorted`

In [ ]:
df = (
    pl.read_csv(csvFile)
    .with_columns(
        pl.col("PassengerId").set_sorted()
    )
)
df["PassengerId"].flags

In the exercises we see what effect `set_sorted` has on performance.

If we transform a column with a sorting operation Polars will update `flags`

In [ ]:
df = (
    pl.read_csv(csvFile)
    .sort("PassengerId")
)
df["PassengerId"].flags

If the data is sorted descending we call:
```python
pl.col("PassengerId").set_sorted(descending=True)
```

### `set_sorted` in an expression
We can use `set_sorted` within an expression. 

For example, if we have a sorted column we can use `set_sorted` to find the `max`

In [ ]:
(
    df
    .select(
        pl.col("PassengerId").set_sorted().max()
    )
)

## Exercises
In the exercises you will develop your understanding of:
- sorting a `DataFrame`
- sorting in an expression
- using `set_sorted`

## Exercise 1: Sorting a `DataFrame`
Sort the `DataFrame` by whether passengers survived and the alphabetical order of the passenger names.

Configure the output to print 10 lines using `pl.Config`

In [ ]:
pl.Config
df = pl.read_csv(csvFile)
df<blank>

Add a column for the `familySize` which is the sum of the number of siblings (`SibSp` columns), the number of parents or children (`Parch` columns) plus one for the passenger themself.

Then sort all of the columns by `familySize` inside an expression

In [ ]:
(
    pl.read_csv(csvFile)
    <blank>
)

## Exercise 2: Using `set_sorted`

We create a random array in Numpy.

We populate a `DataFrame` with:
- a sorted copy of the random array and
- the original random array

In [ ]:
import numpy as np
N = 100_000
randomArray = np.random.randint(0,10,N)
dfSort = pl.DataFrame({
    'sorted':sorted(randomArray),
    'unsorted':randomArray
}
)
dfSort.head(3)

Check to see if Polars thinks either column is sorted

Time how long it takes to find the median of each column

Why is the sorted column column already faster than the unsorted column?

Re-create `dfSort` and tell polars that the `sorted` column is sorted

In [ ]:
N = 1_000_000
randomArray = np.random.randint(0,10,N)
dfSort = (
        pl.DataFrame({
            'sorted':sorted(randomArray),
            'unsorted':randomArray
            }
        )
)
dfSort = (
    dfSort
    <blank>
)

Confirm that Polars knows the `sorted` column is sorted

Compare again how long it takes to find the median of each column

Vary `N` to see how the difference changes with size.

## Solutions

## Solution to Exercise 1
Sort the `DataFrame` by survival and alphabetical order of the passenger names

In [ ]:
pl.Config.set_tbl_rows(10)
(
    pl.read_csv(csvFile)
    .sort(
        ["Survived","Name"]
    )
)

Add a column for the `familySize` which is the sum of the number of siblings (`SibSp` columns), the number of parents or children (`Parch` columns) plus one for the passenger themself.

Then sort all of the columns by `familySize` inside an expression

In [ ]:
(
    pl.read_csv(csvFile)
    .with_columns( 
        (
            pl.col('SibSp') + pl.col('Parch') + 1
        ).alias('familySize')
    )
    .select(
        pl.all().sort_by("familySize")
    )
)

## Solution to exercise 2


Check Polars knows the columns are sorted

In [ ]:
import numpy as np
N = 1_000_000
randomArray = np.random.randint(0,10,N)
dfSort = pl.DataFrame({
    'sorted':sorted(randomArray),
    'unsorted':randomArray
}
)
dfSort.head(3)

In [ ]:
dfSort["sorted"].flags

In [ ]:
dfSort["unsorted"].flags

Time how long it takes to find the median of each column

In [ ]:
%%timeit -n1 -r1 
(
    dfSort
    .select(
        pl.col("sorted").median()
    )
)

In [ ]:
%%timeit -n1 -r1 
(
    dfSort
    .select(
        pl.col("unsorted").median()
    )
)

The sorted column is already faster because finding the median requires a sort and there is no data to be moved around in the sorted column.

Re-create `dfSort` and tell polars that the `sorted` column is sorted

In [ ]:
dfSort = (
    dfSort
    .with_columns(
        pl.col('sorted').set_sorted()
    )
)

Confirm that Polars knows the `sorted` column is sorted

In [ ]:
dfSort["sorted"].flags

Compare again how long it takes to find the median in each column (same as above)

In [ ]:
%%timeit -n1 -r1 
(
    dfSort
    .select(
        pl.col("sorted").median()
    )
)

In [ ]:
%%timeit -n1 -r1 
(
    dfSort
    .select(
        pl.col("unsorted").median()
    )
)